# KPI Reporter

Creates monthly KPI .csv based on data points imported via Eloqua API.

## Data processing

### Import and init modules:

In [1]:
import json
import time
import requests
import datetime
import datascience as ds
from datascience.predicates import are
from IPython.display import clear_output
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

### Data sources

- *eloqua_auth*
- *eloqua_rest*
- *kpi*
- *asset_names*

In [2]:
# Get Eloqua auth key
with open('auth.key') as key:
    eloqua_auth = key.read()
    eloqua_rest = 'https://secure.p06.eloqua.com/api/REST/2.0/'

# KPI segment ID's
kpi = {
    'contact_count': '3666',
    'newsletter_count': '6786',
    'newsletter_profinfo_count': '6791',
    'alert_count': '6792',
    'sent_count': '6793',
    'open_count': '6794',
    'click_count': '6795',
    'form_count': '6796',
    'not_opened_count': '6797',
    'cookie_count': '6798',
    'unsub_form': '892'
}

# List of asset names for eloqua_asset_get()
asset_names = {
    'LP': 'landingPage',
    'Form': 'form',
    'Mail': 'email',
    'Campaign': 'campaign',
    'Program': 'program',
    'Filter': 'contact/filter',
    'Segment': 'contact/segment',
    'Image': 'image'
}

### Helper functions

- __api_request__(*root, call='get', params=None, data=None, files=None*): _Returns response from Eloqua API call_
- __eloqua_segment_refresh__(*segment_id*): _Returns segment count when segment is refreshed (string)_
- __eloqua_get_form_data__(*form_id*): _Returns form data of Form with given ID_
- __segment_array_for__(*category*): _Returns array of results for given KPI category_
- __blacklist_data__(): _Returns number of blacklisted users via Subscription Center unsub form_

In [3]:
def api_request(root, call='get', params=None, data=None, files=None):
    '''
    Arguments:
        root - root URL of API call
        call - either GET/POST/PUT/DELETE
    Returns response from Eloqua API call

    '''

    # Assings correct authorization method
    headers = {'Authorization': eloqua_auth}
    if not files:
        headers['Content-Type'] = 'application/json'

    # Assings correct api call
    if call == 'get':
        response = requests.get(
            root,
            headers=headers,
            params=params)
    elif call == 'post':
        response = requests.post(
            root,
            headers=headers,
            data=data,
            files=files)
    elif call == 'put':
        response = requests.put(
            root,
            headers=headers,
            data=data,
            files=files)
    elif call == 'delete':
        response = requests.delete(root, headers=headers)

    return response


def eloqua_segment_refresh(segment_id):
    '''
    Returns segment count when segment is refreshed (string)
    '''

    # Post refresh queue
    root = eloqua_rest + 'assets/contact/segment/queue/' + segment_id
    queue = api_request(root, call='post')
    queue_data = queue.json()
    queued_at = queue_data['queuedAt']

    # Check if queue has been resolved and segment is refreshed
    root = eloqua_rest + 'assets/contact/segment/' + segment_id + '/count'
    while True:
        time.sleep(10)
        refresh = api_request(root)
        refresh_data = refresh.json()
        calculated_at = refresh_data.get('lastCalculatedAt', '0')
        if int(calculated_at) > int(queued_at):
            break

    return refresh_data['count']


def eloqua_asset_get(asset_id, asset_type, depth=''):
    '''
    Returns name and optionally code of Eloqua asst of given ID
    '''

    # Gets required endpoint
    endpoint = asset_names.get(asset_type)

    # Gets data of requested asset
    root = f'{eloqua_rest}assets/{endpoint}/{asset_id}'
    params = {'depth': 'complete'}
    response = api_request(root, params=params)
    asset_response = response.json()

    # Returns full response
    if depth == 'complete':
        return asset_response

    # Gets name and code of the asset
    name = asset_response['name']
    if asset_type in ['LP', 'Mail']:
        code = asset_response['htmlContent']['html']
    elif asset_type == 'Form':
        code = asset_response['html']

    if asset_type in ['LP', 'Mail', 'Form']:
        return (name, code)
    else:
        return name
    

def eloqua_get_form_data(form_id):
    '''
    Returns form data of Form with given ID
    '''
    all_fills = []
    page = 1

    while True:
        # Gets fills of requested form
        root = f'{eloqua_rest}data/form/{form_id}'
        params = {'depth': 'complete',
                  'count': '100',
                  'page': page}
        response = api_request(root, params=params)
        fills = response.json()

        all_fills.extend(fills['elements'])

        # Stops iteration when full list is obtained
        if fills['total'] - page * int(params.get('count')) < 0:
            break

        # Increments page to get next part of outcomes
        page += 1

    return (all_fills, fills['total'])


def segment_array_for(category):
    '''
    Returns array of results for given KPI category
    '''

    # Refresh segment to get up to date data
    eloqua_segment_refresh(kpi[category])

    # Gets data from segment
    segment_json = eloqua_asset_get(
        kpi[category],
        asset_type='Segment',
        depth='complete'
    )

    # Saves counts to appropriate variables
    for element in segment_json['elements']:
        if element['filter']['name'] == 'ALL+WMR':
            eloqua_and_wmr_contacts = int(element['filter']['count'])
        elif element['filter']['name'] == 'LEG+WMR':
            leg_and_wmr_contacts = int(element['filter']['count'])
        elif element['filter']['name'] == 'PUB+WMR':
            pub_and_wmr_contacts = int(element['filter']['count'])
        elif element['filter']['name'] == 'FIR+WMR':
            fir_and_wmr_contacts = int(element['filter']['count'])
        elif element['filter']['name'] == 'ALL-WMR':
            eloqua_sans_wmr_contacts = int(element['filter']['count'])
        elif element['filter']['name'] == 'LEG-WMR':
            leg_sans_wmr_contacts = int(element['filter']['count'])
        elif element['filter']['name'] == 'PUB-WMR':
            pub_sans_wmr_contacts = int(element['filter']['count'])
        elif element['filter']['name'] == 'FIR-WMR':
            fir_sans_wmr_contacts = int(element['filter']['count'])

    # Calculates others
    oth_and_wmr_contacts = eloqua_and_wmr_contacts - \
        (leg_and_wmr_contacts + pub_and_wmr_contacts + fir_and_wmr_contacts)
    oth_sans_wmr_contacts = eloqua_sans_wmr_contacts - \
        (leg_sans_wmr_contacts + pub_sans_wmr_contacts + fir_sans_wmr_contacts)

    return ds.make_array(
        eloqua_and_wmr_contacts, eloqua_sans_wmr_contacts,
        leg_and_wmr_contacts, leg_sans_wmr_contacts,
        pub_and_wmr_contacts, pub_sans_wmr_contacts,
        fir_and_wmr_contacts, fir_sans_wmr_contacts,
        oth_and_wmr_contacts, oth_sans_wmr_contacts
    )


def blacklist_data():
    '''
    Returns number of blacklisted users via Subscription Center unsub form
    '''
    unsub_form_id = kpi['unsub_form']

    # Gets unsubscription form data
    unsub_list = eloqua_get_form_data(unsub_form_id)[0]

    # Create list of tuples (mail, submit_date)
    blacklisted = []
    for submit in unsub_list:
        for field in submit['fieldValues']:
            email_address = field.get('value', '')
            if '@' in email_address:
                break
        submit_date = int(submit.get('submittedAt', '0'))
        blacklisted.append((email_address, submit_date))

    # Calculate epoch month ago
    timeframe = datetime.datetime.now() + datetime.timedelta(-31)
    epoch_timeframe = int(timeframe.timestamp())

    # Filter only submissions from last month
    last_month_blacklist = [mail for mail,
                            date in blacklisted if date > epoch_timeframe]

    # Deduplicate mails
    last_month_blacklist = list(set(last_month_blacklist))

    return ds.make_array(0, len(last_month_blacklist), 0, 0, 0, 0, 0, 0, 0, len(last_month_blacklist))

### Table creator

- Creates *kpi_table*
- Saves it as .csv

__May need few minutes to process all api calls and data__

» Creating table for KPI data

In [4]:
kpi_table = ds.Table().with_column(
    'Segment',
    ds.make_array(
        'ALL + WMR', 'ALL - WMR',
        'LEG + WMR', 'LEG - WMR',
        'PUB + WMR', 'PUB - WMR',
        'FIR + WMR', 'FIR - WMR',
        'OTH + WMR', 'OTH - WMR'
    )
)

» Gathering data with Eloqua API calls

In [5]:
contact_count = segment_array_for('contact_count')
print('  › Imported contact count data')

  › Imported contact count data


In [6]:
newsletter_count = segment_array_for('newsletter_count')
print('  › Imported newsletter count data')

  › Imported newsletter count data


In [7]:
newsletter_profinfo_count = segment_array_for('newsletter_profinfo_count')
print('  › Imported newsletter profinfo count data')

  › Imported newsletter profinfo count data


In [8]:
alert_count = segment_array_for('alert_count')
print('  › Imported alert count data')

  › Imported alert count data


In [9]:
sent_count = segment_array_for('sent_count')
print('  › Imported sent count data')

  › Imported sent count data


In [10]:
open_count = segment_array_for('open_count')
print('  › Imported open count data')

  › Imported open count data


In [11]:
click_count = segment_array_for('click_count')
print('  › Imported click count data')

  › Imported click count data


In [12]:
form_count = segment_array_for('form_count')
print('  › Imported form count data')

  › Imported form count data


In [13]:
unsub_count = blacklist_data()
print('  › Imported blacklist data')

  › Imported blacklist data


In [14]:
not_opened_count = segment_array_for('not_opened_count')
print('  › Imported not opened count data')

  › Imported not opened count data


In [15]:
cookie_count = segment_array_for('cookie_count')
print('  › Imported cookie count data')

  › Imported cookie count data


» Adding data columns to *kpi_table*

In [16]:
kpi_table = kpi_table.with_columns(
    'Contacts', contact_count,
    'NSL', newsletter_count,
    'NSL Prof', newsletter_profinfo_count,
    'Alert', alert_count,
    'Sent (M)', sent_count,
    'Open (M)', open_count,
    'Click (M)', click_count,
    'Form (M)', form_count,
    'Unsub (M)', unsub_count,
    'Not Opened 10+ (Y)', not_opened_count,
    'Cookie Linked (A)', cookie_count
)

» Saving *kpi_table* to .csv

In [17]:
today = str(datetime.date.today().strftime('%m-%y'))
kpi_table.to_csv(f'WKPL_KPI_{today}.csv')

### Option to load *kpi_table* from .csv file

In [18]:
# Commented out to not interfere with Run all scenario
#kpi_table = ds.Table.read_table('WKPL_KPI_06-18.csv')

## Monthly KPI Data

In [19]:
kpi_table.show()

# Clear the output after checking it is correct
time.sleep(10)
clear_output(wait=True)
print('» Output cleaned')

» Output cleaned
